In [1]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging
import numpy as np
import plotly.graph_objects as go
from PIL import Image

import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.autograd import Variable

from torch.optim import Adam, SGD
import gc

In [2]:
def dta_to_candlestick(data):
    l = len(data)
    # Make candlestick picture
    layout = go.Layout(xaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False,
                                  rangeslider=dict(visible=False)),
                       yaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False),
                       width=512,
                       height=512,
                       paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(data=[go.Candlestick(x=np.linspace(1,l,l),
                                         open=data.Open,
                                         high=data.High,
                                         low=data.Low,
                                         close=data.Close)],
                    layout=layout)
    fig.write_image("images/fig-33.png")

    # Convert to numpy array
    im = Image.open('images/fig-33.png')
    #im = im.resize((300,300),Image.ANTIALIAS)
    data = np.asarray(im)

    # Return the first channel of the image
    return data[:, :, 0]

In [3]:
START = datetime(1980, 1, 1)
END = datetime(2020, 4, 23)

In [4]:
tic = yf.Ticker('MSFT')
hist = tic.history(start=START, end=END)

In [5]:
def dta_transformation(data, est_h):
    # Make sure data has sufficient columns
    assert 'Open' in data.columns
    assert 'High' in data.columns
    assert 'Low' in data.columns
    assert 'Close' in data.columns

    data['lag_close'] = data['Close'].shift(1)
    data['Indicator'] = np.where(data['Close'] > data['lag_close'], 1, 0)

    x = []
    y = []
    for i in range(est_h, data.shape[0]):
        sub_dta = data.iloc[i - est_h:i]

        y_i = data.iloc[i]['Indicator']
        x_i = dta_to_candlestick(sub_dta)

        y.append(y_i)
        x.append(x_i)

        print("{}/{}".format(i - est_h, data.shape[0] - est_h))

    return x, y

In [8]:
hist.iloc[5000:7000]

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2006-01-05,19.75,19.87,19.71,19.77,48245500,0.0,0.0
2006-01-06,19.69,19.77,19.40,19.71,100963000,0.0,0.0
2006-01-09,19.72,19.83,19.60,19.67,55625000,0.0,0.0
2006-01-10,19.52,19.79,19.47,19.77,64921900,0.0,0.0
2006-01-11,19.78,20.06,19.70,19.99,70120700,0.0,0.0
...,...,...,...,...,...,...,...
2013-12-09,33.60,33.87,33.44,33.73,30286000,0.0,0.0
2013-12-10,33.65,33.90,33.13,33.21,37828600,0.0,0.0
2013-12-11,33.17,33.38,32.58,32.78,39853400,0.0,0.0


In [6]:
x, y = dta_transformation(hist.iloc[5000:6000], 10)

C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0/990
1/990
2/990
3/990
4/990
5/990
6/990
7/990
8/990
9/990
10/990
11/990
12/990
13/990
14/990
15/990
16/990
17/990
18/990
19/990
20/990
21/990
22/990
23/990
24/990
25/990
26/990
27/990
28/990
29/990
30/990
31/990
32/990
33/990
34/990
35/990
36/990
37/990
38/990
39/990
40/990
41/990
42/990
43/990
44/990
45/990
46/990
47/990
48/990
49/990
50/990
51/990
52/990
53/990
54/990
55/990
56/990
57/990
58/990
59/990
60/990
61/990
62/990
63/990
64/990
65/990
66/990
67/990
68/990
69/990
70/990
71/990
72/990
73/990
74/990
75/990
76/990
77/990
78/990
79/990
80/990
81/990
82/990
83/990
84/990
85/990
86/990
87/990
88/990
89/990
90/990
91/990
92/990
93/990
94/990
95/990
96/990
97/990
98/990
99/990
100/990
101/990
102/990
103/990
104/990
105/990
106/990
107/990
108/990
109/990
110/990
111/990
112/990
113/990
114/990
115/990
116/990
117/990
118/990
119/990
120/990
121/990
122/990
123/990
124/990
125/990
126/990
127/990
128/990
129/990
130/990
131/990
132/990
133/990
134/990
135/990
136/990
137/990
138/99

In [7]:
x = np.stack(x, axis=2)

In [8]:
x.shape

(512, 512, 990)

In [35]:
im = Image.fromarray(x[:,:,0])
im.show()

In [36]:
x[:,:,0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [37]:
sub_x = x[:,:,568:1568]

In [9]:
y = np.array(y)
y.shape

(990,)

In [24]:
sub_y = y[568:1568]

In [25]:
np.savez_compressed('test_1', x=sub_x, y=sub_y)

In [7]:
8568 % 1000

568

In [27]:
x.shape[2] // 1000

8

In [42]:
for i in range(8):
    sub_x = x[:,:,(1000*i+568):(1000*(i+1)+568)]
    sub_y = y[(1000*i+568):(1000*(i+1)+568)]
    np.savez_compressed('test_{}'.format(i), x=sub_x, y=sub_y)

In [19]:
350000000/1024/1024

333.7860107421875

# Pytorch Machine Learning

In [2]:
from ResNet_CNN import *

import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision
from functools import partial
from collections import OrderedDict

import torchvision.models as models

from torchsummary import summary

In [3]:
model = res_conv1(1, 64)

In [4]:
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBasicBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): LeakyReLU(negative_slope=0.01, inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (

In [5]:
print(summary(model, (1,512,512)))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 128, 64, 64]         --
|    └─Sequential: 2-1                   [-1, 64, 128, 128]        --
|    |    └─Conv2d: 3-1                  [-1, 64, 256, 256]        3,136
|    |    └─BatchNorm2d: 3-2             [-1, 64, 256, 256]        128
|    |    └─LeakyReLU: 3-3               [-1, 64, 256, 256]        --
|    |    └─MaxPool2d: 3-4               [-1, 64, 128, 128]        --
├─ResNetDecoder: 1-2                     [-1, 1]                   --
|    └─AdaptiveAvgPool2d: 2-2            [-1, 128, 1, 1]           --
Total params: 322,843
Trainable params: 322,843
Non-trainable params: 0
------------------------------------------------------------------------------------------
Input size (MB): 1.00
Forward/backward pass size (MB): 64.00
Params size (MB): 1.23
Estimated Total Size (M

## Putting everything together

In [10]:
ticker_list = os.listdir('D:/GitHub/Backtesting/images_npy')

In [9]:
os.getcwd()

'D:\\GitHub\\Backtesting'

In [12]:
ticker_dta = os.listdir('D:/GitHub/Backtesting/images_npy/{}'.format(ticker_list[0]))

In [13]:
ticker_list

['CI', 'DDS', 'IPG', 'LLY', 'MTG', 'NAVI', 'NKE', 'SEQ', 'WY']

In [14]:
ticker_dta

['CI_0.npz',
 'CI_1.npz',
 'CI_2.npz',
 'CI_3.npz',
 'CI_4.npz',
 'CI_5.npz',
 'CI_6.npz',
 'CI_7.npz',
 'CI_8.npz']

In [10]:
temp = np.load('C:/Users/Andyy/Documents/GitHub/Backtesting/images_npy/{}/{}'.format(ticker_list[0], ticker_dta[0]))

In [12]:
dta_x = temp['x']
dta_y = temp['y']

In [11]:
model = res_conv1(1, 64)
lr = 0.0001
optimizer = Adam(model.parameters(), lr=lr)

criterion = nn.BCELoss()

train_losses = []
val_losses = []

In [10]:
loaded = np.load('test.npz')
dta_x = loaded['x']
dta_y = loaded['y']

In [16]:
train_X, train_Y, val_X, val_Y = data_preprocessing(dta_x, dta_y)

((891, 512, 512), (891, 1)) ((99, 512, 512), (99, 1))


In [13]:
dta_y.shape

(990,)

In [25]:
np.savez_compressed('test', x=x, y=y)

In [13]:
def dataLoader(path):
    loaded = np.load(path)
    dta_x = loaded['x']
    dta_y = loaded['y']
    return dta_x, dta_y

In [14]:
for comp in ticker_list:
    ticker_dta = os.listdir('D:/GitHub/Backtesting/images_npy/{}'.format(comp))
    for dta in ticker_dta:
        try:
            path = 'D:/GitHub/Backtesting/images_npy/{}/{}'.format(comp, dta)
            dta_x, dta_y = dataLoader(path)
            print("Train on {}".format(dta))
            # Begin training
            
            train_X, train_Y, val_X, val_Y = data_preprocessing(dta_x, dta_y)
            gc.collect()
            
            train(0)
        except RuntimeError:
            model_path = './cnn_res.pth'
            torch.save(model.state_dict(), model_path)
            print("{} breaks the computer!!!".format(dta))
            break

Train on CI_0.npz
((900, 256, 256), (900, 1)) ((100, 256, 256), (100, 1))
Epoch:  1 	 train loss:  tensor(0.7072, grad_fn=<BinaryCrossEntropyBackward>) 	 val loss:  tensor(0.7103, grad_fn=<BinaryCrossEntropyBackward>)
Train on CI_1.npz
((900, 256, 256), (900, 1)) ((100, 256, 256), (100, 1))
Epoch:  1 	 train loss:  tensor(0.7052, grad_fn=<BinaryCrossEntropyBackward>) 	 val loss:  tensor(0.7158, grad_fn=<BinaryCrossEntropyBackward>)
Train on CI_2.npz
((900, 256, 256), (900, 1)) ((100, 256, 256), (100, 1))
Epoch:  1 	 train loss:  tensor(0.7063, grad_fn=<BinaryCrossEntropyBackward>) 	 val loss:  tensor(0.7024, grad_fn=<BinaryCrossEntropyBackward>)
Train on CI_3.npz
((900, 256, 256), (900, 1)) ((100, 256, 256), (100, 1))
Epoch:  1 	 train loss:  tensor(0.6973, grad_fn=<BinaryCrossEntropyBackward>) 	 val loss:  tensor(0.7002, grad_fn=<BinaryCrossEntropyBackward>)
Train on CI_4.npz
((900, 256, 256), (900, 1)) ((100, 256, 256), (100, 1))
Epoch:  1 	 train loss:  tensor(0.6984, grad_fn=<Binar

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 471859200 bytes. Buy new RAM!
(no backtrace available)

In [20]:
model_path = './cnn_res.pth'
torch.save(model.state_dict(), model_path)

In [21]:
test = res_conv1(1,64)

In [22]:
test.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [14]:
def data_preprocessing(X, Y):
    X = X.reshape((-1, 512, 512)).astype(np.float32)
    Y = Y.reshape((-1, 1)).astype(np.float32)
    train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.1, random_state=42)
    print((train_X.shape, train_Y.shape), (val_X.shape, val_Y.shape))

    train_X = train_X.reshape(-1, 1, 512, 512)
    train_X = torch.from_numpy(train_X)
    train_Y = torch.from_numpy(train_Y)

    val_X =  val_X.reshape(-1, 1, 512, 512)
    val_X = torch.from_numpy(val_X)
    val_Y = torch.from_numpy(val_Y)
    
    return train_X, train_Y, val_X, val_Y

In [22]:
trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [20]:
train_X, train_Y, val_X, val_Y = data_preprocessing(dta_x, dta_y)

((900, 256, 256), (900, 1))((100, 256, 256), (100, 1))


In [17]:
model = res_conv1(1, 64)
lr = 0.0001
optimizer = Adam(model.parameters(), lr=lr)

criterion = nn.BCELoss()

In [18]:
def train(epochs):
    # dataset
    x_train, y_train = Variable(train_X), Variable(train_Y)
    x_val, y_val = Variable(val_X), Variable(val_Y)
 
    optimizer.zero_grad()
    
    output_train = model(x_train)
    output_val = model(x_val)
    
    loss_train = criterion(output_train, y_train.type(torch.float))
    loss_val = criterion(output_val, y_val.type(torch.float))
    
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    
    loss_train.backward()
    optimizer.step()
    
    print('Epoch: ', epochs+1, '\t', 'train loss: ', loss_train, '\t', 'val loss: ', loss_val)

In [23]:
model_path = './cnn_res-1.pth'
torch.save(model.state_dict(), model_path)
print("Breaks the computer!!!")

Breaks the computer!!!


In [19]:
n_epochs = 2
train_losses = []
val_losses = []
tr_loss = 0
try:
    for epochs in range(n_epochs):
        train(epochs)
except RuntimeError:
    model_path = './cnn_res-1.pth'
    torch.save(model.state_dict(), model_path)
    print("Breaks the computer!!!")

Breaks the computer!!!
